# Load file

In [1]:
import json
import string
import nltk
#from nltk.tokenize import TreebankWordTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
import operator
import collections
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import spacy
#%matplotlib inline
#import matplotlib.pyplot as plt
#from networkx.algorithms import community
#import linkcom

In [2]:
# specify dir
Dir = 'D:/Depression_project/data/reddit'

In [3]:
data = []
for i in range(1, 12):
    #with open(Dir+"/randomPosts2017_"+str(i)+".json", mode='r') as file:
    with open(Dir+"/MentalSubmissions2017_"+str(i)+".json", mode='r') as file:
        reader = json.load(file)   # json.load reads whole json file, json.loads reads string only
    for row in reader:
        data.append(row)
print (len(data))

255954


In [116]:
"""""
data = []
with open(Dir+'/'+'randomPosts2017_1.json', mode='r') as file:
#with open(Dir+'/'+'MentalSubmissions2017_1.json', mode='r') as file:
    reader = json.load(file)   # json.load reads whole json file, json.loads reads string only
    for row in reader:
        data.append(row)
print (len(data))

120000


In [175]:
# testing
data[2:4]
#data[1][3]

[{'author': 'Rand0lph_Cart3r',
  'created_utc': 1483230920,
  'id': '5lcmke',
  'num_comments': 5,
  'score': 1,
  'selftext': "The reason that it occurred to me that I may have such repressed memories is unimportant, but after doing some research and reading, I am . . .  a little worried that I may have experienced such abuse at some time in my childhood.  If you will, allow me to list some of the red flags I have uncovered:\n\n-Wearing clothing inappropriate for weather; I always feel at my most comfortable with at least three shirts, a hoodie, and a trench coat, with the addition of a neckerchief over my face and a hat pulled down low, no matter how hot it is.\n\n-Guessing what people want done and doing it without them having to tell me\n\n-Fantasies of rape and domination\n\n-Fantasies of being raped and dominated\n\n-An extremely sensitive gag reflex.\n\n-Lifelong depression and anxiety\n\n-Everything in my childhood, to me, appears to have happened when I was four.\n\n-A prefere

# Extract fields

In [4]:
def extract_field(data, field):
    res = []
    for row in data:
        res.append(row[field])
    print ("The number of "+field+ " is " + str(len(res)))
    return res     

In [136]:
# for comments
author = extract_field(data, 'author')
text = extract_field(data, 'body')
time = extract_field(data, 'created_utc')
ID = extract_field(data, 'id')
link_id = extract_field(data, 'link_id')
parent_id = extract_field(data, 'parent_id')
subr = extract_field(data, 'subreddit')

The number of author is 1200000
The number of body is 1200000
The number of created_utc is 1200000
The number of id is 1200000
The number of link_id is 1200000
The number of parent_id is 1200000
The number of subreddit is 1200000


In [5]:
# for submissions
author = extract_field(data, 'author')
text = extract_field(data, 'selftext')
title = extract_field(data, 'title')
time = extract_field(data, 'created_utc')
ID = extract_field(data, 'id')
subr = extract_field(data, 'subreddit')

The number of author is 255954
The number of selftext is 255954
The number of title is 255954
The number of created_utc is 255954
The number of id is 255954
The number of subreddit is 255954


In [177]:
def time_convert(data):
    res = []
    for t in data:
        parsed_date = datetime.utcfromtimestamp(t)
        year = parsed_date.year
        month = parsed_date.month
        day = parsed_date.day
        time = str(year)+'-'+str(month)+'-'+str(day)
        res.append(time)
    print ("The number of dates is " + str(len(res)))
    return res

In [180]:
time = time_convert(time)

The number of dates is 255954


In [12]:
text[20:30]

["Hey Reddit. I'm not really sure where to go with this. I was clinically diagnosed with major depression with psychotic features, and an anxiety disorder. I think there is more to it though. For reference, I am male by the way. I had a girlfriend at the ripe age of 16 that pretty much forced me into sex at a time I was not ready for. 2 years later, she had constantly grown more and more distant, and used sex to gain leverage over me. Then she admitted after this time that she had been cheating on me for a year. Obviously, I ended it there and haven't seen the bitch since. I'm worried, however, that some sort of schizophrenic personality disorder is affecting me. I'm obviously no psychiatrist, and shouldn't self diagnose, but I'm realizing things I haven't told my psychiatrists. Whenever I get close with someone, I usually sabotage it and just watch it burn. Then when I lose contact with people it makes me extraordinarily lonely. I see things that aren't there. I have severe paranoia o

# Construct a dataframe

### For random posts

In [139]:
# for comments
d = {'id': ID, 'text': text, 'author': author, 'time': time, 'subreddit': subr}
# for submissions
#d = {'id': ID, 'title': title, 'text': text, 'author': author, 'time': time, 'subreddit': subr}
df1 = pd.DataFrame(data=d)
df1.head()

,author,id,subreddit,text,time
0,aregulardude,dg97ncw,askscience,Well it depends how you're spacecraft is desig...,2017-4-14
1,therealflyingtoastr,dfqve6r,MLS,The comparison with DCU has literally nothing ...,2017-4-2
2,stifmeister917,dfwty5q,PS4,Yo,2017-4-6
3,Aborawatabinov,dg6p9ii,ifyoulikeblank,Make sure you listen to all three volumes of J...,2017-4-12
4,raffobaghdo,dfwf3eg,AskReddit,Skiplagged.com,2017-4-6


In [140]:
# delete duplicated posts
df = df1.drop_duplicates(subset=['id'], keep='first')
print (len(df['id']))

1199205


In [141]:
# remove deleted or removed posts
df = df[df['text'] != '[deleted]']
print (len(df['id']))
df = df[df['text'] != '[removed]']
print (len(df['id']))

1149424
1121689


In [142]:
# exclude depression related subreddits
print (len(set(df['subreddit'])))

dep_list = ['mentalhealth','depression','depression_help','bipolar','BipolarReddit','ADHD','Anxiety', \
            'Schizophrenia','SuicideWatch','selfharm','Mentalillness','traumatoolbox']

for subr in dep_list:
    df = df[df['subreddit'] != subr]
    print ("After excluding "+subr+", the total number of posts is "+str(len(df['id'])))

24209
After excluding mentalhealth, the total number of posts is 1121620
After excluding depression, the total number of posts is 1120971
After excluding depression_help, the total number of posts is 1120955
After excluding bipolar, the total number of posts is 1120739
After excluding BipolarReddit, the total number of posts is 1120687
After excluding ADHD, the total number of posts is 1120388
After excluding AnxietySchizophrenia, the total number of posts is 1120388
After excluding SuicideWatch, the total number of posts is 1120119
After excluding selfharm, the total number of posts is 1120084
After excluding Mentalillness, the total number of posts is 1120084
After excluding traumatoolbox, the total number of posts is 1120084


In [156]:
df['text'][:10]
print (len(df['text']))

1120084


In [151]:
f = open(Dir+'/'+'randomSubreddits2017.txt', 'w')
for row in df['text'][:10]:
    f.write(row)
    f.write('###')
f.close()

### For depression posts

In [181]:
# for comments
#d = {'id': ID, 'text': text, 'author': author, 'time': time, 'subreddit': subr}
# for submissions
d = {'id': ID, 'title': title, 'text': text, 'author': author, 'time': time, 'subreddit': subr}
df1 = pd.DataFrame(data=d)
df1.head()

,author,id,subreddit,text,time,title
0,Mr_Anomalous,5lchbv,mentalhealth,"I don't know how many of you read it, but a fe...",2017-1-1,I don't think that was just an anxiety attack.
1,[deleted],5lcj3c,mentalhealth,[deleted],2017-1-1,Controlling anger?
2,Rand0lph_Cart3r,5lcmke,mentalhealth,The reason that it occurred to me that I may h...,2017-1-1,Are these signs of repressed memories of child...
3,enjetant,5lcovl,mentalhealth,Sometimes I don't really feel like a participa...,2017-1-1,Feeling like a spectator to your emotions
4,[deleted],5lcs0u,mentalhealth,[deleted],2017-1-1,Why shouldn't I eat only 1-2 times a day?


In [192]:
df = df1[(df1['text'] != '[deleted]') & (df1['title'] != '[deleted]')]
print (len(df['id']))
df = df[(df['text'] != '[removed]') & (df['title'] != '[removed]')]
print (len(df['id']))
#df = df[(df['text'] != '[deleted]') & (df['title'] != '[removed]')]
#df = df[(df['text'] != '[removed]') & (df['title'] != '[deleted]')]

194819
177855


In [193]:
# include specific depression related subreddits
print (len(set(df['subreddit'])))

dep_list = ['mentalhealth','depression','depression_help','bipolar','BipolarReddit','ADHD','Schizophrenia','Mentalillness']

df = df[df['subreddit'].isin(dep_list)]
print (len(df['id']))

9
145046


In [197]:
df['text_new'] = df['title'] + ' ' + df['text']

In [198]:
df[10000:10005]

,author,id,subreddit,text,time,title,text_new
14770,bestrival,5p0dup,bipolar,i feel like every other night im crying and on...,2017-1-19,Am i ruining my life,Am i ruining my life i feel like every other n...
14771,berfica,5p0fw1,bipolar,I'm depressed. Was in the hospital half of las...,2017-1-20,Things are bad...,Things are bad... I'm depressed. Was in the ho...
14772,steellaw11,5p0hnh,bipolar,I suffer from both and when my new PCP treated...,2017-1-20,"Anyone else also have ADHD, need in your input...","Anyone else also have ADHD, need in your input..."
14773,sleepyval,5p0ooh,bipolar,"""My body is mine. My brain sometimes isn't."" -...",2017-1-20,Just a quick quote i thought was relevant.,"Just a quick quote i thought was relevant. ""My..."
14774,petites_pattes,5p0qxd,bipolar,I feel like I (and some of us) tend to only ta...,2017-1-20,SUCCESSES,SUCCESSES I feel like I (and some of us) tend ...


# Preprocess data

In [194]:
# define a function to part of speech a list of document
# PoS type: noun, pronoun, verb, adverb, adjective, conjunction, preposition, interjection, -PRON- refers to personal pronoun
def pos(data):  # the input is a document
    outputdata = []
    import spacy 
    import en_core_web_sm
    nlp = en_core_web_sm.load()
    for line in data:
        doc = nlp(line)
        newline = []
        for token in doc:
            if len(token) < 18:
                if token.pos_ == 'NOUN' or token.pos_ == 'PROPN' or token.pos_ == 'VERB' or token.pos_ == 'ADJ' \
                or token.pos_ == 'ADV':
                    w = token.lemma_
                    if w == '-PRON-':
                        w = str(token).lower()
                    stemmer = nltk.stem.PorterStemmer() 
                    w = stemmer.stem(w)   
                    newline.append(w)
        outputdata.append(newline)    
    return outputdata

In [195]:
# randomly select n posts
def randomSelect(seed, data, n):
    selected_texts = []
    np.random.seed(seed) 
    selected_ids = np.random.choice(len(data), n) 
    for i in selected_ids:
        selected_texts.append(data[i])
    return selected_texts

In [170]:
# for random posts
data = list(df['text'])
selected_texts = randomSelect(1, data, 100000)
print (len(selected_texts))

100000


In [199]:
# for depr posts
data = list(df['text_new'])
selected_texts = randomSelect(100, data, 100000)
print (len(selected_texts))

100000


In [200]:
# for random posts
import datetime
start = datetime.datetime.now().replace(microsecond=0)
cleaned_texts = pos(selected_texts)
print(len(selected_texts))
end = datetime.datetime.now().replace(microsecond=0)
print(end-start)

100000
2:53:29


In [201]:
cleaned_texts[:2]

[['anybodi', 'els', 'look', 'find', 'reason', 'live', 'can', 'not', 'find'],
 ['bad', 'part', 'be', 'have', 'live', 'crazi', 'fuck', 'world']]

In [202]:
# save data
f = open(Dir+'/'+'mentalSubreddits2017_selected.txt',"w")
#f = open(Dir+'/'+'randomSubreddits2017_selected.txt',"w")
for row in cleaned_texts:
    try:
        sent = ' '.join(row)
        f.write(sent+'\n')
    except:
        continue
f.close()

In [155]:
# read data
text = []
f = open(Dir+'/'+'randomSubreddits2017.txt',"r")
#f = open(Dir+'/'+'randomPosts2017_cleaned.txt',"r")
d = f.read().split('###')
for line in d:
    text.append(line)
print (len(text))
text[1]

11


"The comparison with DCU has literally nothing to do with the amounts. He's contrasting how DCU's public funding falls generally equally on the richer and poorer sections of the metro area while the STL bid has the taxes largely paid by the poorer sections of town while leaving the richer white suburbs untouched."

In [5]:
parent_ids = []
link_ids = []
ids = []
for row in data:
    parent_ids.append(row['parent_id'])
    link_ids.append(row['link_id'])
    ids.append(row['id'])
print (len(parent_ids))
print (len(link_ids))
print (len(ids))

466252
466252
466252


In [45]:
count = 0
for i in ids:
    if i == 'czivrnx':
        print (texts[count],parent_ids[count])
    count+=1


&gt; "Animus debe mutare, non caelum"

This was said by Seneca some 2000 years ago, referring precisely to depression. It roughly translates to "you must change yourself, not the sky you live under".

I personally get the urge to run away quite often, but I always realise I will not find any more happiness on the other side of the world than I do here, if I don't "fix" myself first. t3_43k4vd


In [46]:
count = 0
for i in ids:
    if i == '43k4vd':
        print (texts[count],parent_ids[count])
    count+=1

In [52]:
mydata = []
count = 0
for i in parent_ids:
    if i.startswith('t1_'):
        mydata.append(texts[count])
    count+=1
print (len(mydata))

204813


In [53]:
mydata[:10]

['&gt; I always realise I will not find any more happiness on the other side of the world than I do here, if I don\'t "fix" myself first.\n\nHave you tested this theory?',
 "I would be good enough when, people don't just look through me, when I can wake up in the morning and feel accomplished, when I can look my mom in her eyes; and know she is actually proud of me. I'll feel good enough when I can look myself in the mirror and tell myself I am.",
 "I've been really hesitant to. I've already called them on him once, but he of course used it against me and said it was a shitty thing to do despite the fact I thought he was actually going to kill himself. It did force him to get a diagnosis and therapist and medication, but that was in September and he's run out of meds and free sessions. I know I should, but it's daunting",
 "It was on the same day as my one year evaluation. We had recently gone through a change of positions in the company so I went from not having someone to report to t

# Extract text

In [4]:
texts = []
for row in data:  
    texts.append(row['body'])
print (len(texts))

print (texts[1])

466252
Great question. I think the short answer to your question is basically no, there's nothing really as effective as prescription antidepressants except maybe exercise but not even. I know that is a sucky answer and I wish you could get some great miracle drug as your answer instead.

Would you be interested in sharing with us why you want something nonprescription instead of getting a prescription from a doctor?


# Preprocess data

In [59]:
# check posts to see the deleted or removed posts
#texts[1000:2000]

In [5]:
# remove deleted or removed posts
filtered_texts = []
for text in texts:
    if text == '[deleted]' or text == '[removed]':
        pass
    else:
        filtered_texts.append(text)

print (len(filtered_texts))

407459


In [60]:
#filtered_texts[:1000]

In [147]:
"""""
# test
print (sample_data[0][2][1])
tokenizer = nltk.TreebankWordTokenizer()
tokenizer.tokenize(sample_data[0][2][1])

I'm never going to convince you so I'm not trying to.  You have yet to share any studies, and any you do share will only reflect legal gun ownership.  And gun crime is perpetrated primarily by those who do not possess their guns legally.

The number I've seen show that a third of those with post graduate degrees are firearms owners, so to say once you have an education you don't want to own a firearm is asinine.

But by all means, just keep prattling on about your poor pa back in west pennsyltucky whose a crotchety ol bastard and a lout.


['I',
 "'m",
 'never',
 'going',
 'to',
 'convince',
 'you',
 'so',
 'I',
 "'m",
 'not',
 'trying',
 'to.',
 'You',
 'have',
 'yet',
 'to',
 'share',
 'any',
 'studies',
 ',',
 'and',
 'any',
 'you',
 'do',
 'share',
 'will',
 'only',
 'reflect',
 'legal',
 'gun',
 'ownership.',
 'And',
 'gun',
 'crime',
 'is',
 'perpetrated',
 'primarily',
 'by',
 'those',
 'who',
 'do',
 'not',
 'possess',
 'their',
 'guns',
 'legally.',
 'The',
 'number',
 'I',
 "'ve",
 'seen',
 'show',
 'that',
 'a',
 'third',
 'of',
 'those',
 'with',
 'post',
 'graduate',
 'degrees',
 'are',
 'firearms',
 'owners',
 ',',
 'so',
 'to',
 'say',
 'once',
 'you',
 'have',
 'an',
 'education',
 'you',
 'do',
 "n't",
 'want',
 'to',
 'own',
 'a',
 'firearm',
 'is',
 'asinine.',
 'But',
 'by',
 'all',
 'means',
 ',',
 'just',
 'keep',
 'prattling',
 'on',
 'about',
 'your',
 'poor',
 'pa',
 'back',
 'in',
 'west',
 'pennsyltucky',
 'whose',
 'a',
 'crotchety',
 'ol',
 'bastard',
 'and',
 'a',
 'lout',
 '.']

In [65]:
"""""
# define a function to preprocess data
def preprocess(sent):
    tokenizer = nltk.TreebankWordTokenizer()
    tokens = tokenizer.tokenize(sent)
    filtered_tokens = [w for w in tokens if not w in stopwords.words('english') and not w in string.punctuation and\
                       not w.isdigit() and not w.startswith("http")]
    return filtered_tokens

def clean(data):
    cleaned_data = []
    for row in data:       
        cleaned_data.append(preprocess(row[1])) 
     
    # delete long messy words
    for text in cleaned_data:    
        for word in text:
            if len(word) > 20:
                text.remove(word)
                
    # need to do twice            
    for text in cleaned_data:    
        for word in text:   
            if len(word) > 20:
                text.remove(word) 
                
    return cleaned_data                  

In [6]:
# define a function to preprocess data
def preprocess(sent):
    sent = sent.translate(str.maketrans('', '', string.punctuation))    # this is for python 3
    tokens = sent.lower().split()
    filtered_tokens = [w for w in tokens if not w in stopwords.words('english') and not w.isdigit()]
    return filtered_tokens

In [15]:
cleaned_texts = []
for text in filtered_texts:    
    cleaned_texts.append(preprocess(text))
print (len(cleaned_texts))

KeyboardInterrupt: 

In [16]:
print (len(cleaned_texts))
cleaned_texts[:1000]

220566


[['younger',
  'thought',
  'life',
  'could',
  'different',
  'new',
  'place',
  'different',
  'people',
  'ended',
  'realizing',
  'though',
  'gonna',
  'story',
  'anywhere',
  'else',
  'problem',
  'attitude',
  'let',
  'things',
  'affect',
  'agree',
  'teddy',
  'r'],
 ['great',
  'question',
  'think',
  'short',
  'answer',
  'question',
  'basically',
  'theres',
  'nothing',
  'really',
  'effective',
  'prescription',
  'antidepressants',
  'except',
  'maybe',
  'exercise',
  'even',
  'know',
  'sucky',
  'answer',
  'wish',
  'could',
  'get',
  'great',
  'miracle',
  'drug',
  'answer',
  'instead',
  'would',
  'interested',
  'sharing',
  'us',
  'want',
  'something',
  'nonprescription',
  'instead',
  'getting',
  'prescription',
  'doctor'],
 ['gt',
  'always',
  'realise',
  'find',
  'happiness',
  'side',
  'world',
  'dont',
  'fix',
  'first',
  'tested',
  'theory'],
 ['would',
  'good',
  'enough',
  'people',
  'dont',
  'look',
  'wake',
  'mornin

# Part of speech

In [47]:
"""""
# need to load models beofre using it 
# python -m spacy download en
import spacy 
import en_core_web_sm
nlp = en_core_web_sm.load()
doc = nlp(u'Apple is not looking at buying U.K. startup for $1 billion.')
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
          token.shape_, token.is_alpha, token.is_stop)

Apple apple PROPN NNP nsubj Xxxxx True False
is be VERB VBZ aux xx True True
not not ADV RB neg xxx True True
looking look VERB VBG ROOT xxxx True False
at at ADP IN prep xx True True
buying buy VERB VBG pcomp xxxx True False
U.K. u.k. PROPN NNP compound X.X. False False
startup startup NOUN NN dobj xxxx True False
for for ADP IN prep xxx True True
$ $ SYM $ quantmod $ False False
1 1 NUM CD compound d False False
billion billion NUM CD pobj xxxx True False
. . PUNCT . punct . False False


In [213]:
""""
cleaned_data = []
for line in cleaned_data2[:2]:
    cleaned_data.append(' '.join(line))
cleaned_data     

[u'wow never even made connection thats brutal lol irony terrible',
 u'dont really know probably ones pulling strings chetto puppet']

In [153]:
cleaned_data2 = []
for dat in cleaned_data:
    a = []
    for line in dat:
        a.append(' '.join(line))
    cleaned_data2.append(a)

In [154]:
cleaned_data2[2][:3]

['yes know thats democrats need come cohesive plan confront complain koch brothers afp',
 'boston dont know nyc though',
 'far right trying return country guilded age late 19th century life great wealthy white men pretty fucking horrible everyone else']

In [114]:
# check
stemmer = nltk.stem.PorterStemmer()  # work for verb
stemmer.stem('consider')

'consid'

In [110]:
# check
stemmer = nltk.stem.WordNetLemmatizer()    # work for some plural form non
stemmer.lemmatize('wolves') 

'wolf'

In [155]:
# need to load models before using it the first time
# python -m spacy download en

# define a function to part of speech a list of document

def pos(data):  #the input is a document
    outputdata = []
    import spacy 
    import en_core_web_sm
    nlp = en_core_web_sm.load()
    for line in data:
        doc = nlp(line)
        newline = []
        for token in doc:
            if token.pos_ == 'NOUN' or token.tag_ == 'NOUN' or token.pos_ == 'VERB' or token.pos_ == 'ADJ' or token.pos_ == 'ADV':
                newline.append(token.lemma_)
        outputdata.append(newline)
    
    return outputdata

In [156]:
data_pos = []
for dat in cleaned_data2:
    pos(dat)
    data_pos.append(pos(dat))
                    
print(len(data_pos)) 

43


In [122]:
# check
#data_pos[0]

In [37]:
# save data
f = open(Dir+'/'+'depression2016_cleaned_posts.txt',"w")
for row in cleaned_texts:
    for w in row:
        try:
            f.write(w+' ')
        except:
            pass
    f.write('###')
f.close()

#####################################################################

In [17]:
corpus = []
for text in cleaned_texts:
    for word in text:
        corpus.append(word)
print (len(corpus))

6303895


In [18]:
count = collections.Counter(corpus)
#count

In [19]:
uncommon = []
for k, v in list(count.items()):
    if v <= 5:
        uncommon.append(k)
        del count[k]   # remove the word from the whole corpus
print (len(uncommon))

74533


In [20]:
cleaned = []
for text in cleaned_texts:
    cleaned_sent = []
    for word in text:
        if word not in set(uncommon):
            cleaned_sent.append(word)
    cleaned.append(cleaned_sent)

In [21]:
print (len(cleaned))
cleaned[:10]

10


[['younger',
  'thought',
  'life',
  'could',
  'different',
  'new',
  'place',
  'different',
  'people',
  'ended',
  'realizing',
  'though',
  'gonna',
  'story',
  'anywhere',
  'else',
  'problem',
  'attitude',
  'let',
  'things',
  'affect',
  'agree',
  'teddy',
  'r'],
 ['great',
  'question',
  'think',
  'short',
  'answer',
  'question',
  'basically',
  'theres',
  'nothing',
  'really',
  'effective',
  'prescription',
  'antidepressants',
  'except',
  'maybe',
  'exercise',
  'even',
  'know',
  'sucky',
  'answer',
  'wish',
  'could',
  'get',
  'great',
  'miracle',
  'drug',
  'answer',
  'instead',
  'would',
  'interested',
  'sharing',
  'us',
  'want',
  'something',
  'instead',
  'getting',
  'prescription',
  'doctor'],
 ['gt',
  'always',
  'realise',
  'find',
  'happiness',
  'side',
  'world',
  'dont',
  'fix',
  'first',
  'tested',
  'theory'],
 ['would',
  'good',
  'enough',
  'people',
  'dont',
  'look',
  'wake',
  'morning',
  'feel',
  'acco

In [34]:
dat = []
f = open(Dir+'/'+'depression2016_cleaned_posts.txt',"r")
d = f.read().split('###')
for line in d:
    dat.append(line)
len(dat)

220567

In [35]:
dat[0]

'younger thought life could different new place different people ended realizing though gonna story anywhere else problem attitude let things affect agree teddy r '

In [39]:
dat[1].split()

['great',
 'question',
 'think',
 'short',
 'answer',
 'question',
 'basically',
 'theres',
 'nothing',
 'really',
 'effective',
 'prescription',
 'antidepressants',
 'except',
 'maybe',
 'exercise',
 'even',
 'know',
 'sucky',
 'answer',
 'wish',
 'could',
 'get',
 'great',
 'miracle',
 'drug',
 'answer',
 'instead',
 'would',
 'interested',
 'sharing',
 'us',
 'want',
 'something',
 'nonprescription',
 'instead',
 'getting',
 'prescription',
 'doctor']